In [48]:
import os
os.environ['OPENAI_API_BASE'] = 'https://oneapi.xty.app/v1'
os.environ["OPENAI_API_KEY"] = "sk-9QT7CG36EPZc0snHF7FfEcCf113c4d678e48C8D97fCa8cFf"

In [49]:
# 设置分词器：当llama_index需要分词时，会使用指定的tiktoken编码器
from llama_index.core import Settings
# tiktoken
import tiktoken
Settings.tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo").encode

In [50]:
# attach to the same event-loop
import nest_asyncio
nest_asyncio.apply()

In [51]:
from llama_index.core import KeywordTableIndex # 创建基于关键词的索引（通过关键词快速检索文档中的信息）
from llama_index.core import SimpleDirectoryReader # 读取位于特定目录中的文档（文档可能用于构建索引或进行文本分析）
from llama_index.llms.openai import OpenAI

In [52]:
# define LLM
llm = OpenAI(
    temperature=0,  # temperature=0 表示在生成文本时不加入随机性。在语言模型中，temperature是一个用于控制生成文本随机性的参数。temperature为 0 通常意味着模型会生成确定性最高的输出，即最可能的预测。
    model="gpt-3.5-turbo",  # 指定要使用的 OpenAI 模型的名称
    max_tokens=512 # 设置生成文本的最大令牌数。tokens是语言模型处理的文本的基本单位，通常对应于单词或字符。这个参数限制了模型一次可以生成的文本量，以避免生成过长的输出。
)

In [54]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 句子嵌入（Sentence Embeddings）是将句子转化为固定大小的向量表示，以便于进行各种NLP任务，如语义相似度比较、文本分类等。sentence-transformers 是一个流行的Python库，用于生成句子嵌入。
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-zh-v1.5"
)

In [55]:
# 内置加载器，用于从本地目录加载各种文件类型
documents = SimpleDirectoryReader("/Users/skbx1060/Documents/RAG/RAGAS/annotation_data/").load_data()

In [64]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
# parse nodes
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

# build index
index = VectorStoreIndex(nodes,llm=llm)

In [42]:
# # build index
# index = KeywordTableIndex.from_documents(documents, llm=llm) # LLM实例用于增强索引的构建过程(通过理解文档的语义内容来改进关键词的提取和索引

In [65]:
query_engine = index.as_query_engine()

In [58]:
# get response from query
response = query_engine.query("暴发性心肌炎的定义是什么？")
print(response)

暴发性心肌炎定义为急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病，通常是一个临床诊断，需要结合临床表现、实验室及影像学检查进行综合分析。


Evaluation

In [66]:
eval_questions = [
    "暴发性心肌炎的定义是什么？",
    "UA有哪些临床表现？",
    "暴发性心肌炎在病程早期一般需要与哪些心血管系统疾病和其他可以引起相应临床表现的疾病进行区别？",
    "针对ATTR-CA的治疗分为哪些治疗？"
]

eval_answers = [
    "一般将暴发性心肌炎定义为急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病",
    " UA有以下临床表现：①静息性心绞痛：心绞痛发作在休息时，并且持续时间通常在20 min以上；②初发心绞痛：1个月内新发心绞痛，可表现为自发性发作与劳力性发作并存，疼痛分级在Ⅲ级阱上；③恶化劳力型心绞痛：既往有心绞痛病史，近1个月内心绞痛恶化加重，发作次数频繁、时间延长或痛阈降低(心绞痛分级至少增加1级，或至少达到Ⅲ级)。",
    "1. 冠心病;2. 病毒性肺炎;3. 脓毒血症性心肌炎;4. 应激性心肌病（ Takotsubo综合征）;5. 普通急性心肌炎;6. 非病毒性暴发性心肌炎",
    "针对ATTR-CA的治疗分为对症治疗和病因治疗，其靶点治疗包括抑制TTR合成的药物和稳定TTR的药物。"
]

eval_answers = [[a] for a in eval_answers]

In [67]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

In [72]:
from ragas.llama_index import evaluate
result = evaluate(query_engine, metrics, eval_questions, eval_answers)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 20/20 [00:37<00:00,  1.88s/it]


In [73]:
print(result)

{'faithfulness': 0.9167, 'answer_relevancy': 0.6809, 'context_precision': 1.0000, 'context_recall': 1.0000, 'harmfulness': 0.0000}


In [74]:
result.to_pandas()

,question,answer,contexts,ground_truths,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,暴发性心肌炎的定义是什么？,暴发性心肌炎通常被定义为一种急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病。这个定义强调了...,[诊断\n\n一般将暴发性心肌炎定义为急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病，因此...,[一般将暴发性心肌炎定义为急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病],一般将暴发性心肌炎定义为急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病,0.666667,0.929774,1.0,1.0,0
1,UA有哪些临床表现？,UA的临床表现包括静息性心绞痛、初发心绞痛和恶化劳力型心绞痛。静息性心绞痛的特点是发作在休息...,[(一)临床表现\n\n1. UA有以下临床表现：①静息性心绞痛：心绞痛发作在休息时，并且持...,[ UA有以下临床表现：①静息性心绞痛：心绞痛发作在休息时，并且持续时间通常在20 min以...,UA有以下临床表现：①静息性心绞痛：心绞痛发作在休息时，并且持续时间通常在20 min以上...,1.000000,0.000000,1.0,1.0,0
2,暴发性心肌炎在病程早期一般需要与哪些心血管系统疾病和其他可以引起相应临床表现的疾病进行区别？,"In the early stages of the disease, it is gene...",[诊断\n\n一般将暴发性心肌炎定义为急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病，因此...,[1. 冠心病;2. 病毒性肺炎;3. 脓毒血症性心肌炎;4. 应激性心肌病（ Takots...,1. 冠心病;2. 病毒性肺炎;3. 脓毒血症性心肌炎;4. 应激性心肌病（ Takotsu...,1.000000,0.826517,1.0,1.0,0
3,针对ATTR-CA的治疗分为哪些治疗？,针对ATTR-CA的治疗分为对症治疗和病因治疗，其靶点治疗包括抑制TTR合成的药物和稳定TT...,[**治疗**\n\n针对ATTR-CA的治疗分为对症治疗和病因治疗，其靶点治疗包括抑制TT...,[针对ATTR-CA的治疗分为对症治疗和病因治疗，其靶点治疗包括抑制TTR合成的药物和稳定T...,针对ATTR-CA的治疗分为对症治疗和病因治疗，其靶点治疗包括抑制TTR合成的药物和稳定TT...,1.000000,0.967227,1.0,1.0,0


In [75]:
result_new = result.to_pandas()
# print(type(result))
# print(type(result_new))
# print(len(result))
# print(len(result_new))
# print(result_new.columns)
for i in range(len(result_new)):
    print(result_new['question'][i],':')
    print('----------------------------------------------------------------------------------')
    print('answer:---------------',result_new['answer'][i])
    print('contexts:-------------',result_new['contexts'][i])
    print('ground_truth:---------',result_new['ground_truths'][i])
    print('\n')

暴发性心肌炎的定义是什么？ :
----------------------------------------------------------------------------------
answer:--------------- 暴发性心肌炎通常被定义为一种急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病。这个定义强调了疾病的急性起病和临床表现，特别是在短时间内出现严重的心脏功能障碍。暴发性心肌炎的诊断往往依赖于临床表现、实验室检查以及影像学检查的综合分析。
contexts:------------- ['诊断\n\n一般将暴发性心肌炎定义为急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病，因此暴发性心肌炎更多是一个临床诊断而非组织学或病理学诊断，因而诊断需要结合临床表现、实验室及影像学检查综合分析。 当出现发病突然，有明显病毒感染前驱症状尤其是全身乏力、不思饮食继而迅速出现严重的血液动力学障碍、实验室检测显示心肌严重受损、超声心动图可见弥漫性室壁运动减弱时，即可临床诊断暴发性心肌炎。\n\n鉴别诊断：由于暴发性心肌炎可累及多器官和系统，临床表现严重且具有多样性，病情进展迅速，在病程早期常需要使用一些检查以排除其他疾病。包括心血管系统疾病和其他可以引起相应临床表现的疾病。\n\n1. 冠心病：急性大面积心肌梗死可出现肺淤血水肿导致循环衰竭、休克，心肌标志物可显著升高，暴发性心肌炎需与其进行鉴别。主要通过冠状动脉造影进行鉴别，另外冠心病患者彩色超声心动图可见明显心肌局限性运动异常。\n2. 病毒性肺炎：重症肺炎合并脓毒血症休克时也可出现心肌标志物轻度一过性升高，但随休克及血氧饱和度的纠正而显著改善。\n3. 脓毒血症性心肌炎：严重细菌感染休克时毒性损害也可致心肌损伤而加重休克，并可出现明显心脏抑制性表现。早期出现的感染灶及血自细胞早期即显著升高及其他全身表现有助于鉴别。\n4.'
 '脓毒血症性心肌炎：严重细菌感染休克时毒性损害也可致心肌损伤而加重休克，并可出现明显心脏抑制性表现。早期出现的感染灶及血自细胞早期即显著升高及其他全身表现有助于鉴别。\n4. 应激性心肌病（ Takotsubo综合征） ： 又称心尖球形综合征，好发于绝经期后女性，有胸痛、心电图 ST-T改变以及心肌损伤标志物升高。 常有强烈精神刺激等诱因。左心室造影可见节段性室壁运动异常，